In [1]:
# 모듈 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# 0) 시드 생성 : 항상 같은 결과를 보장하기 위함
torch.manual_seed(1)

In [4]:
# 1) 데이터 정의
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # x데이터 : 6x2 matrix
y_data = [[0], [0], [0], [1], [1], [1]] # y데이터 : 6x1 matrix

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [5]:
# pytorch exp 테스트
print('e^1 equals : ', torch.exp(torch.FloatTensor([1])))

e^1 equals :  tensor([2.7183])


In [9]:
# 2) 모델 가정
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 1way
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
print(hypothesis)
print(hypothesis.shape)

# 2way
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


In [10]:
# 손실 측정 테스트
-(y_train[0] * torch.log(hypothesis[0]) + (1-y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

In [14]:
# 3) 손실 측정
# 1way
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
cost = losses.mean()
print(losses)
print(cost)

# 2way
cost = F.binary_cross_entropy(hypothesis, y_train)
print(cost)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)
tensor(0.6931, grad_fn=<MeanBackward0>)
tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)


In [23]:
# 전체 코드

# 모듈 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 0) 시드 생성
torch.manual_seed(1)

# 1) 데이터 정의
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # x데이터 : 6x2 matrix
y_data = [[0], [0], [0], [1], [1], [1]] # y데이터 : 6x1 matrix

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 2) 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 3) Optimizer 설정
optimizer = torch.optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 4) H(x) 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    
    # 5) cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # 6) cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 7) 출력
    if epoch % 100 == 0:
        print('Epoch : {:4d}/{}, Cost : {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch :    0/1000, Cost : 0.693147
Epoch :  100/1000, Cost : 0.134722
Epoch :  200/1000, Cost : 0.080643
Epoch :  300/1000, Cost : 0.057900
Epoch :  400/1000, Cost : 0.045300
Epoch :  500/1000, Cost : 0.037261
Epoch :  600/1000, Cost : 0.031672
Epoch :  700/1000, Cost : 0.027556
Epoch :  800/1000, Cost : 0.024394
Epoch :  900/1000, Cost : 0.021888
Epoch : 1000/1000, Cost : 0.019852


In [24]:
# 모델 평가 테스트
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [28]:
# 7) 모델 평가
prediction = hypothesis >= torch.FloatTensor([0.5])
correct_prediction = prediction.float() == y_train # prediction : ByteTensor

print(prediction[:5])
print(y_train[:5])
print(correct_prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])
tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [29]:
# class 상속
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1) # 로지스틱 회귀이므로 input_dim=8, output_dim=1.
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
model = BinaryClassifier()

In [37]:
# 전체 코드

# 모듈 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 0) 시드 생성
torch.manual_seed(1)

# 1) 데이터 정의
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # x데이터 : 6x2 matrix
y_data = [[0], [0], [0], [1], [1], [1]] # y데이터 : 6x1 matrix

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 2) 모델 초기화
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1) # 뭐냐 이건
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
model = BinaryClassifier()

# 3) Optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 4) H(x) 계산
    hypothesis = model(x_train)
    
    # 5) cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # 6) cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 7) 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch : {:4d}/{}, Cost : {:.6f}, Accuracy : {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x2 and 8x1)